<a href="https://colab.research.google.com/github/Fas96/-/blob/master/NBADATAANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Presentation**

In [ ]:
import streamlit as st
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
from PIL import Image
import plotly.graph_objects as go

image = Image.open("basicG/img.png")
st.image(image, use_column_width=True)

st.title('NBA Player Statistics')

st.markdown("""This app performs simple webscraping of NBA player stats data!
* **Python libraries:** base64, pandas, streamlit
* **Data source:** [Basketball-reference.com](https://www.basketball-reference.com/).
""")

st.sidebar.header('User Input Features')
selected_year = st.sidebar.selectbox('Year', list(reversed(range(1950, 2020))))a


# Web scraping of NBA player stats
@st.cache
def load_data(year):
    url = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + "_per_game.html"
    html = pd.read_html(url, header=0)

    df = html[0]
    # print(df.describe())
    raw = df.drop(df[df.Age == 'Age'].index)  # Deletes repeating headers in content
    raw = raw.fillna(0)
    playerstats = raw.drop(['Rk'], axis=1)
    return playerstats


playerstats = load_data(selected_year)

# Sidebar - Team selection
sorted_unique_team = sorted(playerstats.Tm.unique())
selected_team = st.sidebar.multiselect('Team', sorted_unique_team, sorted_unique_team)

# Sidebar - Position selection
unique_pos = ['C', 'PF', 'SF', 'PG', 'SG']
selected_pos = st.sidebar.multiselect('Position', unique_pos, unique_pos)

# Filtering data
df_selected_team = playerstats[(playerstats.Tm.isin(selected_team)) & (playerstats.Pos.isin(selected_pos))]

st.header('Display Player Statistic of Teams')
st.write('Data Dimension: ' + str(df_selected_team.shape[0]) + ' rows and ' + str(df_selected_team.shape[1]) + ' columns.')
st.dataframe(df_selected_team.astype(str))


# Download NBA player stats data
# https://discuss.streamlit.io/t/how-to-download-file-in-streamlit/1806
def filedownload(df):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # strings <-> bytes conversions
    href = f'<a href="data:file/csv;base64,{b64}" download="playerstats.csv">Download CSV File</a>'
    return href


st.markdown(filedownload(df_selected_team), unsafe_allow_html=True)

col1, col2, col3,col4,col5 = st.columns([0.4, 0.2, 0.3,0.2,0.2])

# Heatmap
if col1.button('Intercorrelation Heatmap'):
    st.header('Intercorrelation Matrix Heatmap')
    df_selected_team.to_csv('output.csv', index=False)
    df = pd.read_csv('output.csv')

    corr = df.corr()
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize=(7, 5))
        ax = sns.heatmap(corr, mask=mask, vmax=1, square=True)
        st.pyplot(f)

# Sigmoid
if col2.button('Sigmoid'):
    st.header("Sigmoid")
    df_selected_team.to_csv('output.csv', index=False)
    df = pd.read_csv('output.csv')

    b = df.iloc[:, 1:].values

    f = []
    for aa in b:
        f.append(aa[3:])
    array2D = np.array(f)
    fig = px.scatter(x=array2D.flatten(), y=array2D.flatten()[::-1], )
    st.plotly_chart(fig)

# Pie Chart
if col3.button('Age Distribution'):
    st.header("Age Distribution")
    df_selected_team.to_csv('output.csv', index=False)
    df = pd.read_csv('output.csv')

    array2D = np.array(df.iloc[:, 2:3].values)

    unique, frequency = np.unique(array2D.flatten(), return_counts = True)
    print(unique,"----",frequency)
    #The plot
    fig = go.Figure(go.Pie(labels = unique, values = frequency,hoverinfo = "label+percent",textinfo = "value"))
    st.plotly_chart(fig)


# Scattered
if col4.button('Points'):
    st.header("Points Distribution")
    df_selected_team.to_csv('output.csv', index=False)
    df = pd.read_csv('output.csv')

    array2D = np.array(df.iloc[:, -2:-1].values)
    array3D = np.array(df.iloc[:, -1:].values)
    print(array3D)
    fig = px.scatter(
        x = array2D.flatten(),
        y = array3D.flatten(),
    )

    # unique, frequency = np.unique(array2D.flatten(), return_counts = True)
    # print(unique,"----",frequency)
    #The plot
    # fig = go.Figure(go.Pie(labels = unique, values = frequency,hoverinfo = "label+percent",textinfo = "value"))
    st.plotly_chart(fig)



# Scattered
if col5.button('Age & Points '):
    st.header("Age and Points  Distribution")
    df_selected_team.to_csv('output.csv', index=False)
    df = pd.read_csv('output.csv')
    array3D = np.array(df.iloc[:, -1:].values)


    fig = px.bar(
        df, #Data Frame
        x = "PTS", #Columns from the data frame
        y = "Age",
        title = "Line frame"
    )

    st.plotly_chart(fig)